In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from tqdm import tqdm
import cv2
import os
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle as sk_shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
from src.data import get_data
from src.data import get_image_value

Using TensorFlow backend.


In [5]:
dim = (224,224)
model_type = 'Mobilenet'
bw = False

train_paths, train_labels = get_data('train')
train_images = np.array([get_image_value(i, dim, bw, model_type) for i in train_paths])
train_dict = dict(images = train_images, labels = train_labels)

TRAIN Value Counts
0    31
1    21
dtype: int64


In [13]:
test_paths, test_labels = get_data('test')
test_images = np.array([get_image_value(i, dim, bw, model_type) for i in test_paths])
test_dict = dict(images = test_images, labels = test_labels)

TEST Value Counts
0    8
1    6
dtype: int64


In [14]:
train_images.shape

(52, 224, 224, 3)

In [15]:
x_train = train_images
y_train = train_labels
x_test = test_images
y_test = test_labels

In [18]:
from src.model import MobileNet

In [24]:
model = MobileNet(input_shape = train_images.shape[1:])

(224, 224, 3)


In [25]:
augmentation =ImageDataGenerator(rotation_range = 20, width_shift_range = .2, height_shift_range = .2, 
                                                       horizontal_flip = True, shear_range = .15, 
                                 fill_mode = 'nearest', zoom_range = .15)

In [26]:
augmentation.fit(x_train)

In [27]:
mobilenet_history = model.model.fit_generator(augmentation.flow(x_train, y_train, batch_size = model.batch_size),
                                              epochs = model.epochs, 
                                              callbacks = [model.early_stopping, model.model_checkpoint, model.lr_plat],
                                              validation_data = (x_test, y_test),
                                              verbose= 1)
                                              

Epoch 1/1000
1/1 [==============================] - 4s 4s/step - loss: 0.9500 - accuracy: 0.5000 - val_loss: 0.6273 - val_accuracy: 0.7143

Epoch 00001: val_loss improved from inf to 0.62732, saving model to ./data/models/mobilnet/mobilenet.h5
Epoch 2/1000
1/1 [==============================] - 1s 726ms/step - loss: 0.7386 - accuracy: 0.5385 - val_loss: 0.5684 - val_accuracy: 0.7857

Epoch 00002: val_loss improved from 0.62732 to 0.56840, saving model to ./data/models/mobilnet/mobilenet.h5
Epoch 3/1000
1/1 [==============================] - 1s 706ms/step - loss: 0.6920 - accuracy: 0.6538 - val_loss: 0.5403 - val_accuracy: 0.8571

Epoch 00003: val_loss improved from 0.56840 to 0.54026, saving model to ./data/models/mobilnet/mobilenet.h5
Epoch 4/1000
1/1 [==============================] - 1s 709ms/step - loss: 0.6267 - accuracy: 0.6731 - val_loss: 0.5236 - val_accuracy: 0.8571

Epoch 00004: val_loss improved from 0.54026 to 0.52359, saving model to ./data/models/mobilnet/mobilenet.h5
Epo

In [23]:
# val_accuracy: 0.7857

In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from keras import regularizers
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import load_model

In [8]:
def get_mobilenet(dim):
    model = Sequential()
    optimizer = Adam(lr = .0005)
    baseModel = MobileNetV2(weights="imagenet", include_top=False,
        input_tensor=Input(shape=dim))
    
    model.add(baseModel)
    model.add(AveragePooling2D(pool_size=(7, 7)))
    model.add(Flatten(name="flatten"))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.6))
    model.add(Dense(2, activation="sigmoid", name = 'Output'))
    
    for layer in baseModel.layers:
        layer.trainable = False
        
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model



In [9]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=5, min_delta = .00075)
model_checkpoint = ModelCheckpoint(f'ModelWeights/Mobilenet_Masks.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 64